# Modello LSMT (Long Short-Term Memory) 

Primo step split del dataset

In [ ]:
import glob
import os



In [ ]:
INPUT_DIR = '../Pre-Elaborazione Dati/Dataset'
all_stitched_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

# 2. Definisci la suddivisione 16-4-4 (8-2-2 mesi)
# (Assicurati che i tuoi file siano ordinati cronologicamente, 
#  'blocco_000', 'blocco_001', ecc. come abbiamo fatto)

TRAIN_FILES = all_stitched_files[0:16]  # Primi 16 blocchi (8 mesi)
VAL_FILES = all_stitched_files[16:20] # Successivi 4 blocchi (2 mesi)
TEST_FILES = all_stitched_files[20:24] # Ultimi 4 blocchi (2 mesi)

print(f"File di Training: {len(TRAIN_FILES)}")
print(f"File di Validazione: {len(VAL_FILES)}")
print(f"File di Test: {len(TEST_FILES)}")

# Stampa un esempio
print(f"\nPrimo file di Train: {os.path.basename(TRAIN_FILES[0])}")
print(f"Primo file di Val: {os.path.basename(VAL_FILES[0])}")
print(f"Primo file di Test: {os.path.basename(TEST_FILES[0])}")

In [ ]:
import pandas as pd
import glob
import os
import gc
from collections import Counter

# --- 1. CONFIGURAZIONE ---
INPUT_DIR = '/home/al3th3ia/Scrivania/Cybersecurity/Detecting-Trajectory-Spoofing-Attacks-on-AIS/Progetto/Pre-Elaborazione Dati/Dataset' 

all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if len(all_files) == 0:
    print(f"ERRORE CRITICO: Nessun file .parquet trovato in '{INPUT_DIR}'")
else:
    print(f"Trovati {len(all_files)} file .parquet in '{INPUT_DIR}'")

TRAIN_FILES = all_files[0:16]
IPERPARAMETRI_TIMESTEP = [10, 15, 20, 25, 30, 40]

# --- FASE 1: Scansione Unificata (Anti-RAM Crash) ---
print(f"\nFASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...")

total_counts_counter = Counter()
total_ids_set = set()

for file_path in TRAIN_FILES:
    print(f"  Processando {os.path.basename(file_path)}...")
    try:
        df = pd.read_parquet(file_path, columns=['TrajectoryID'])
        
        # 1. Metodo Set (affidabile)
        total_ids_set.update(df['TrajectoryID'].unique())
        
        # 2. Metodo Counter (NUOVA VERSIONE CORRETTA)
        # Invece di .groupby(), che fallisce, diamo in pasto
        # l'intera colonna al Counter. È molto più efficiente.
        total_counts_counter.update(df['TrajectoryID'])
        
        del df
        gc.collect()
    except Exception as e:
        print(f"    ERRORE nel leggere {file_path}: {e}")
        
print("\n" + "---" * 20)
print("FASE 1: Scansione completata.")

# --- FASE 2: Verifica e Analisi ---
print("FASE 2: Verifica e Analisi.")

conteggio_set = len(total_ids_set)
conteggio_counter = len(total_counts_counter)

print(f"\VERIFICA CONTEGGI:")
print(f"ID Unici (metodo 'set'):     {conteggio_set:,}")
print(f"ID Unici (metodo 'Counter'): {conteggio_counter:,}")

# Ora DEVONO corrispondere
if conteggio_set != conteggio_counter:
    print("ERRORE LOGICO: I conteggi non corrispondono. C'è ancora un problema.")
else:
    print("\nSUCCESSO! I conteggi corrispondono.")
    
    # Ora che siamo sicuri, procediamo con l'analisi
    # Estraiamo le LUNGHEZZE dal counter
    all_lengths = list(total_counts_counter.values())
    totale_traiettorie_analisi = conteggio_counter

    print(f"\nRisultati dell'analisi (su {totale_traiettorie_analisi:,} traiettorie totali di TRAINING):")
    print("-" * 75)
    print(f"{'Window Size':<15} | {'Traiettorie Usabili':<20} | {'Traiettorie Scartate':<20} | {'% Usabili':<10}")
    print("-" * 75)

    for timestep in IPERPARAMETRI_TIMESTEP:
        traiettorie_usabili = sum(1 for length in all_lengths if length >= timestep)
        traiettorie_scartate = totale_traiettorie_analisi - traiettorie_usabili
        percentuale_usabili = (traiettorie_usabili / totale_traiettorie_analisi) * 100
        print(f"{timestep:<15} | {traiettorie_usabili:<20} | {traiettorie_scartate:<20} | {percentuale_usabili:<10.2f}%")

    print("-" * 75)
    print("Analisi completata.")

<>:53: SyntaxWarning: invalid escape sequence '\V'
<>:53: SyntaxWarning: invalid escape sequence '\V'
/tmp/ipykernel_11288/674630576.py:53: SyntaxWarning: invalid escape sequence '\V'
  print(f"\VERIFICA CONTEGGI:")


Trovati 25 file .parquet in '/home/al3th3ia/Scrivania/Cybersecurity/Detecting-Trajectory-Spoofing-Attacks-on-AIS/Progetto/Pre-Elaborazione Dati/Dataset'

FASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...
  Processando blocco_000-segmentato.parquet...
  Processando blocco_001-segmentato.parquet...
  Processando blocco_002-segmentato.parquet...
  Processando blocco_003-segmentato.parquet...
  Processando blocco_004-segmentato.parquet...
  Processando blocco_005-segmentato.parquet...
  Processando blocco_006-segmentato.parquet...
  Processando blocco_007-segmentato.parquet...
  Processando blocco_008-segmentato.parquet...
  Processando blocco_009-segmentato.parquet...
  Processando blocco_010-segmentato.parquet...
  Processando blocco_011-segmentato.parquet...
  Processando blocco_012-segmentato.parquet...
  Processando blocco_013-segmentato.parquet...
  Processando blocco_014-segmentato.parquet...
  Processando blocco_015-segmentato.parquet...

-----------------------------------